Create a custom kernel to use the python and python libraries installed in this particular virtualenvironment to provide a complete and isolated package


In [40]:
import os
import time
import re
import pyowm
from enum import Enum
from slackclient import SlackClient
from config import SLACK_BOT_TOKEN, OWM_API_KEY

In [41]:
# instantiate Slack client
slack_client = SlackClient(SLACK_BOT_TOKEN)
# starterbot's user ID in Slack: value is assigned after the bot starts up
starterbot_id = None



owm = pyowm.OWM(OWM_API_KEY)  # 

# constants
RTM_READ_DELAY = 1 # 1 second delay between reading from RTM
EXAMPLE_COMMAND = "do"
NOT_IMPLEMENTED = "Sure...write some more code then I can do that!"
GREETINGS = "Welcome <@{user_id}> to the channel <#{channel}>! Hope you have a good time here!"
MENTION_REGEX = "^<@(|[WU].+?)>(.*)"
WEATHER_REGEX = "*weather*"
WEATHER="weather"

In [42]:
class labels(Enum):
    example="example"
    greeting="greeting"
    default="default"
    weather="weather in city"
    weather_my_loc = "weather in my location"
    dnt_knw_loc="My Apologies! I don't have the capability to guess your current location yet! I'm still learning. Please tell me your city, I'll remember it the next time. Trust me!"

In [43]:
msg_to_resp = {"example":NOT_IMPLEMENTED,
               "greeting":GREETINGS,
               "default":"Not sure what you mean. Try *{}*.".format(EXAMPLE_COMMAND),
               "weather":WEATHER}

In [44]:
city_name_userid = dict()

In [45]:
# API Wrappers

def post_message(msg, channel, ts=None, user=None):
    if user:
        return slack_client.api_call("chat.postEphemeral", channel=channel
                                     ,text=msg, user=user)
    else:
        return slack_client.api_call("chat.postMessage", channel=channel
                                     ,text=msg, thread_ts=ts)

def update_message(msg, channel, ts):
    return slack_client.api_call("chat.update", channel=channel
                                 ,text=msg, ts=ts)

def delete_message(msg, channel, ts):
    return slack_client.api_call("chat.delete", channel=channel
                                 , ts=ts)

def add_reaction(emoji, channel, timestamp):
    return slack_client.api_call("reactions.add", channel=channel,
                                 name=emoji,timestamp=timestamp)

def remove_reaction(emoji, channel, timestamp):
    return slack_client.api_call("reactions.remove", channel=channel,
                                 name=emoji,timestamp=timestamp)


In [46]:
# API Wrappers for OWM

def get_weather_and_location(place=None, cityid=None, coords=None, zipcode=None):
    """
    Extract the weather and location object for a given location attribute.
    """
    if cityid:
        o = owm.weather_at_id(cityid)
    elif coords:
        o = owm.weather_at_coords(coords) 
    elif zipcode:
        o = owm.weather_at_zip_code(zipcode)
    else:
        o = owm.weather_at_place(place)

    return o.get_weather(), o.get_location()

In [47]:
def classify_message(msg, userid=None):
    if msg.startswith(EXAMPLE_COMMAND):
        return labels.example.name
    elif msg.startswith("Welcome"):
        return labels.greeting.name
    elif re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather ', msg, re.I|re.M):
        return labels.weather.name
    elif re.search(r'weather', msg, re.I|re.M):
        return labels.weather_my_loc.name
    else:
        return labels.default.name

In [48]:
def get_city_name(msg, user_id=None):
    """
    Extract the city name for the message.
    """
    if re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather ', msg, re.I|re.M):
        mtch = re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather ', msg, re.I|re.M)
        city = next(filter(lambda x: x is not None, mtch.groups()), None)
    else:
        city = city_name_userid.get(user_id, labels.dnt_knw_loc.name)
    return city


In [58]:
def get_weather_in_city(city):
    try:
        w, l = get_weather_and_location(place=city)

        status = w.get_detailed_status()
        temp = w.get_temperature(unit='celsius')
        temp_cur = temp.get('temp', 0)
        temp_max = temp.get('temp_max', float('inf'))
        temp_min = temp.get('temp_min', float('-inf'))
        unit = 'Celsius'

        weather = """The weather in {city} is {status}. The current temperature is {cur} {unit} \
    and is expected to have a high of {max} {unit} and a low of {min} {unit}""".format(city=city, status=status,
                                                                                           cur=temp_cur, max=temp_max,
                                                                                           min=temp_min, unit=unit)
    except:
        weather = """I'm sorry; I'm still learning. I'm currently unable to find the weather of {city}. May be the city has other names? Have you tried them?""".format(city=city)
    return weather

In [50]:
def parse_bot_commands(slack_events):
    """
        Parses a list of events coming from the Slack RTM API to find bot commands.
        If a bot command is found, this function returns a tuple of command and channel.
        If its not found, then this function returns None, None.
    """
    for event in slack_events:
        
        if event["type"] == "member_joined_channel" and event["user"] != starterbot_id:
            user_id, channel = event["user"], event["channel"]
            message = "Welcome!!"
            return message, channel, user_id
        
        if event["type"] == "message" and not "subtype" in event:
            user_id, message = parse_direct_mention(event["text"])
            if user_id == starterbot_id:
                return message, event["channel"], user_id
    return None, None, None

def parse_direct_mention(message_text):
    """
        Finds a direct mention (a mention that is at the beginning) in message text
        and returns the user ID which was mentioned. If there is no direct mention, returns None
    """
    matches = re.search(MENTION_REGEX, message_text)
    # the first group contains the username, the second group contains the remaining message
    return (matches.group(1), matches.group(2).strip()) if matches else (None, None)

def handle_command(command, channel, user_id=None):
    """
        Executes bot command if the command is known
    """
    
    msg = classify_message(command)
    
    # Use the classified message to obtain the corresponding response
    response = msg_to_resp.get(msg, None)
    
    if msg == labels.greeting.name:
        response = response.format(user_id=user_id, channel=channel)
    elif msg in [labels.weather.name, labels.weather_my_loc.name]:
        city = get_city_name(command, user_id)
        response = get_weather_in_city(city)

    # Sends the response back to the channel
    post_message(msg=response or default_response, channel=channel)

In [ ]:
if __name__ == "__main__":
    if slack_client.rtm_connect(with_team_state=False):
        print("Starter Bot connected and running!")
        
        # Read bot's user ID by calling Web API method `auth.test`
        starterbot_id = slack_client.api_call("auth.test")["user_id"]
        while True:
            command, channel, user_id = parse_bot_commands(slack_client.rtm_read())
            if command:
                handle_command(command, channel, user_id)
            time.sleep(RTM_READ_DELAY)
    else:
        print("Connection failed. Exception traceback printed above.")

In [ ]:
# mtch = re.search(r'weather in (.*) | (.*)\'s weather | (.*) weather', 'BaNgalore weather', re.I|re.M)
mtch = re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather', 'weather in', re.I|re.M)

In [ ]:
if not mtch:
    print(True)

In [ ]:
def classify_message(msg, userid=None):
    if msg.startswith(EXAMPLE_COMMAND):
        return labels.example.name
    elif msg.startswith(GREETINGS):
        return labels.greeting.name
    elif re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather ', msg, re.I|re.M):
        mtch = re.search(r'(.*) weather|weather in (.*)|(.*)\'s weather ', msg, re.I|re.M)
        city = next(filter(lambda x: x is not None, mtch.groups()), None)
    elif re.search(r'weather', msg, re.I|re.M):
        city = get_city_name(userid)
        if not city:
            return labels.dnt_knw_loc.name 
    else:
        return labels.default.name
        
    

In [ ]:
import sys
sys.executable

In [51]:
o = owm.weather_at_zip_code(zipcode='94014', country='US')

w = o.get_weather()

In [56]:
w.__dir__()

['__lt__',
 '_temperature',
 '__delattr__',
 'get_detailed_status',
 '__setattr__',
 '__getattribute__',
 '_humidex',
 '__str__',
 'get_dewpoint',
 '_humidity',
 'get_sunrise_time',
 '_sunrise_time',
 '_rain',
 '_reference_time',
 '_snow',
 '_to_DOM',
 '_clouds',
 'get_rain',
 '__reduce_ex__',
 '_heat_index',
 '_detailed_status',
 'get_pressure',
 '__new__',
 'to_JSON',
 '__le__',
 'to_XML',
 '_sunset_time',
 'get_weather_code',
 '_dewpoint',
 'get_humidex',
 '__dict__',
 '__class__',
 '__doc__',
 '__repr__',
 '__hash__',
 '_weather_icon_name',
 '__subclasshook__',
 'get_sunset_time',
 '__gt__',
 'get_snow',
 '__ne__',
 'get_humidity',
 '__init__',
 '__eq__',
 '_visibility_distance',
 '__reduce__',
 '__sizeof__',
 'get_weather_icon_name',
 '__format__',
 'get_clouds',
 '_status',
 '__ge__',
 'get_visibility_distance',
 '_pressure',
 'get_wind',
 'get_reference_time',
 '__module__',
 'get_status',
 '_weather_code',
 '_wind',
 '__dir__',
 'get_heat_index',
 '__weakref__',
 'get_temperatu

In [57]:
w.get_detailed_status()

'clear sky'

In [ ]:
w.get_temperature(unit='celsius')

In [ ]:
o1 = owm.weather_at_place('New York')

w1 = o1.get_weather()

In [ ]:
w1.get_temperature(unit='celsius')
